In [ ]:
Hts = [0, 0.25, 0.5]

from spirit import simulation, state,quantities, hamiltonian,parameters,geometry,configuration,system,io
import numpy as np
import os
import matplotlib.pyplot as plt
import multiprocessing as mp
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd


output_interval = 2 #Interval at which spin configuration files are saved
fn = "dipolar_arr"
prefix = "DDI_exp_14_G0p00005_Ht10p0"

iterations_per_step = 1 #Take this many Metropolis iterationss per lattice site between each check for convergence
converge_threshold = 0.01 #Fractional change in magnetization between steps to accept convergence
converge_max = 20 #Maximum number of steps to take before moving on
mu = 7
dim = 4
concentration = 55

#with state.State("input/test_Ising_largelattice.cfg") as p_state:

def plot_loop(Ht):
    with state.State(f"input/LHF_DDI_glass_14_{concentration}_tunnel_{dim}.cfg") as p_state:
        types = geometry.get_atom_types(p_state)
        nos = types.size
        print(f"Sites: {nos}  Spins: {nos+np.sum(types)}")
        locs = geometry.get_positions(p_state)
        np.savetxt("output/"+prefix+"atom_locs.csv",locs,delimiter=",")
        np.savetxt("output/"+prefix+"atom_types.csv",types,delimiter=",")
    #    write_config(p_state,prefix)
        parameters.mc.set_metropolis_cone(p_state,use_cone=True,cone_angle=0.00000001,use_adaptive_cone=False)
        parameters.mc.set_metropolis_spinflip(p_state,True)
        parameters.mc.set_tunneling_gamma(p_state, tunneling_gamma=0.00012)

        parameters.mc.set_temperature(p_state, 0.000000001)

        #We'll evaluate convergence after enough Metropolis steps to hit each site twitce on average
        parameters.mc.set_iterations(p_state,iterations_per_step*types.size,iterations_per_step*types.size)

        #Initialize in the saturated state
        configuration.plus_z(p_state)

    ## Import the DDI interaction arrays
    #    path_arr_x = os.path.join("input/", fn +"_DDI_x.npy")
    #    path_arr_y = os.path.join("input/", fn +"_DDI_y.npy")
    #    path_arr_z = os.path.join("input/", fn +"_DDI_z.npy")
        path_arr_x = os.path.join(f"dipolar_interaction_matrices_reordered/{dim}_{dim}_{dim}/", fn +"_x.npy") #fn = dipolar_arr
        path_arr_y = os.path.join(f"dipolar_interaction_matrices_reordered/{dim}_{dim}_{dim}/", fn +"_y.npy")
        path_arr_z = os.path.join(f"dipolar_interaction_matrices_reordered/{dim}_{dim}_{dim}/", fn +"_z.npy")

        if os.path.exists(path_arr_x) and os.path.exists(path_arr_y) and os.path.exists(path_arr_z):
            print("loading DDI interaction data.")
            DDI_interaction_x = np.load(path_arr_x)
            DDI_interaction_y = np.load(path_arr_y)
            DDI_interaction_z = np.load(path_arr_z)
        else:
            print("DDI files not found")
    #        break

    #Check that the size of the DDI arrays matches NOS (extracted from the types array above).
        if (nos != DDI_interaction_x.shape[0]) or (nos != DDI_interaction_y.shape[0]) or (nos != DDI_interaction_z.shape[0]) :
            print("Size mismatch between DDI and spin array")
    #        break
    #Filter out any vacant sites
        vacancies_idx = np.where(types == -1)
        locs[:,0][vacancies_idx] = 0
        locs[:,1][vacancies_idx] = 0
        locs[:,2][vacancies_idx] = 0

        i = 0
        Hz = 0
        # for i,Hz in enumerate(fields_hyst):

        print(f'{Hz:.3f}', concentration)

        Hmag = np.sqrt(Hz*Hz + Ht*Ht)
        hamiltonian.set_field(p_state,Hmag,(Ht,0,Hz)) #Inside set_field, the vector is normalized, so we don't have to do that here

        spins = system.get_spin_directions(p_state)  #Get the current spin state to update the DDI fields from the Ewald sum
        spins[:,2][vacancies_idx] = 0   #For LHF, we only care about Sz, but zero out the moments for vacancy site

        #Calculate the DDI field components, and then send to the SPIRIT engine. DDI interaction calculations are in Oe, SPIRIT
        #uses T, so need to scale accordingly.
        #Get the field at each spin. Only care about DDI due to z-spin so use spins[:,2]
        DDI_field_x = np.matmul(DDI_interaction_x,spins[:,2]) *7/1e4
        DDI_field_y = np.matmul(DDI_interaction_y,spins[:,2]) *7/1e4
        DDI_field_z = np.matmul(DDI_interaction_z,spins[:,2]) *7/1e4

        #Pass into SPIRIT
        DDI_field_interleave = np.ravel(np.column_stack((DDI_field_x,DDI_field_y,DDI_field_z)))
        system.set_DDI_field(p_state,n_atoms=nos,ddi_fields=DDI_field_interleave)
        # print(f"X: mean: {np.mean(DDI_field_x):.4e} Std. Dev: {np.std(DDI_field_x):.4e}")
        # print(f"Y: mean: {np.mean(DDI_field_y):.4e} Std. Dev: {np.std(DDI_field_y):.4e}")
        print(f"Z: mean: {np.mean(DDI_field_z):.4e} Std. Dev: {np.std(DDI_field_z):.4e}")

        #Check convergence, same as old hysteresis_loop. But METHOD_MC now uses tunnelling since we set tunnel flag to 1 in cfg files
        for j in range(converge_max) :
            simulation.start(p_state, simulation.METHOD_MC, single_shot=False) #solver_type=simulation.MC_ALGORITHM_METROPOLIS
            simulation.stop(p_state)
            if j == 0 :
                m_temp = quantities.get_magnetization(p_state)[2]
            else :
                m_prev = m_temp
                m_temp = quantities.get_magnetization(p_state)[2]
 #               ratio = abs((m_temp-m_prev)/m_prev)
                ratio = abs((m_temp-m_prev)/mu)
                print(f"Iteration: {j:d}, Convergence: {ratio:.4f}, M_z: {m_temp:.4f}")
                if ratio<converge_threshold :
                    break


    return df




In [ ]:
if __name__ == '__main__':
    # spin_concentrations = [85, 95, 15, 25, 35, 45, 55, 65, 75]
    n_cycles = 1
    conc = 55
    # spin_concentrations = [conc] * n_cycles

    # Assuming fields_hyst and mz are already defined

    with mp.Pool(processes=3) as pool:
        results = pool.map(plot_loop, Hts)

In [4]:
from spirit import simulation, state,quantities, hamiltonian,parameters,geometry,configuration,system,io
import numpy as np
import os
import matplotlib.pyplot as plt
import multiprocessing as mp
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
concentration = 25
dim = 4

In [8]:
with state.State(f"input/LHF_DDI_glass_14_{concentration}_tunnel_{dim}.cfg") as p_state:
    print(type(p_state))

2025-07-06 09:44:43  [  ALL  ] [ALL ] [--] [--]  =====================================================
                                                 ========== Spirit State: Initialising... ============
                                                 ==========     Version:  2.2.0
                                                 ==========     Revision: 57bb4f9adb68c
2025-07-06 09:44:43  [  ALL  ] [ALL ] [--] [--]  Config file: "input/LHF_DDI_glass_14_25_tunnel_4.cfg"
2025-07-06 09:44:43  [  ALL  ] [ALL ] [--] [--]  =====================================================
2025-07-06 09:44:43  [WARNING] [ IO ] [--] [--]  Keyword 'lattice_rng_seed' not found. Using Default: -332800635
2025-07-06 09:44:43  [WARNING] [ IO ] [--] [--]  4 atom types, iatom=0 atom type=0 concentration=0.25
2025-07-06 09:44:43  [WARNING] [ IO ] [--] [--]  Keyword 'mc_seed' not found. Using Default: -1083652157
2025-07-06 09:44:43  [WARNING] [ IO ] [--] [--]  Keyword 'ema_sparse' not found. Using Default: fals